# 최종 웹 크롤링 코드 
- 
- 결과 데이터
    - shop_name     # 쇼핑몰 출처
    - display_name   # 쇼핑몰에 노출되는 판매명
    - brand_name    # 브랜드
    - original_price  # 원가
    - sale_price      # 판매가
    - image_url       # 이미지 링크 (jpg 확인 필요)
    - ~~rating~~      # 평점 (보류 중)
    - rank           # 순위
- 특이사항
    - 할인가만 있고 원가는 없을때, 원가 = 할인가 처리함
        - 할인이 없을때 해당 경우 있는 것으로 확인
    - 브랜드명을 가져올때, 판매자명만 있는 경우 판매자명을 가져오게 함
        (개인 사업자 분들 같음. 예시 제품 : https://item.gmarket.co.kr/Item?goodscode=681524167)
    - 판매자명이 "스타배송"인 경우있음. gmarket에서 따로 빠르게 배송되는 상품들을 제품들을 모아서 만들어둔 것으로 보임
        - 이런 제품의 브랜드명은 None으로 처리했음
        (예시 제품:https://item.gmarket.co.kr/Item?goodscode=3259256560)

## step1. 제품 순위 페이지에서 제품목록 가져오기

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # 백그라운드 실행
chrome_options.add_argument("--no-sandbox")  # 리눅스에서 필요
chrome_options.add_argument("--disable-dev-shm-usage")  # 메모리 문제 해결
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화
chrome_options.add_argument("--window-size=1920,1080")  # 창 크기 설정
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")  # User Agent 설정
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 봇 감지 우회
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# 봇 감지 추가 우회
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

#-------------------------------------------------------
# 지마켓 쇼핑몰 페이지와 연결
driver.get("https://www.gmarket.co.kr/n/best?groupCode=100000005&subGroupCode=200000037")
driver.implicitly_wait(10)
page_source = driver.page_source
driver.quit()

In [ ]:
soup = BeautifulSoup(page_source, 'html.parser')
list__best = soup.find(class_="list__best")
# class="list-item"인 요소들만 찾기
best_item_list = list__best.select('[class="list-item"]')

items_info_list = []
shop_name = "gmarket"
for item in best_item_list:
    sale_price = int(item.find(attrs={'class':"box__price-seller"}).select('.text.text__value')[0].text.replace(',', ''))
    org_price  = sale_price
    if item.find(attrs={'class':"box__price-original"}):
        # 원가 정보가 있으면 사용
        org_price = int(item.find(attrs={'class':"box__price-original"}).select('.text.text__value')[0].text.replace(',', ''))
    
    item_info = {
        'shop_name' : shop_name,
        'item_url'  : item.find(attrs={'class':'link'}).get('href').strip(),
        'display_name' : item.find(attrs={'class':'box__item-title'}).text.strip(), 
        'original_price' : org_price, 
        'sale_price' : sale_price,
        'image_url' : item.find(attrs={'class':"box__thumbnail"}).find('img').get('src'),
        'rank' : int(item.find(attrs={'class':'box__label-rank'}).text.strip()),
    }
    items_info_list.append(item_info)

In [4]:
item_info

{'shop_name': 'gmarket',
 'item_url': 'https://item.gmarket.co.kr/Item?goodscode=2904434225',
 'display_name': '(아이허브) 3개X 오메가800 피쉬오일 EPA/DHA800mg 30젤 해외직구',
 'original_price': 46900,
 'sale_price': 30010,
 'image_url': '//gdimg.gmarket.co.kr/2904434225/still/300?ver=1759111302',
 'rank': 200}

In [5]:
items_info_list

[{'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=4272567157',
  'display_name': '활기력 20ml 16병 선물포장 (쇼핑백동봉)',
  'original_price': 42000,
  'sale_price': 29900,
  'image_url': '//gdimg.gmarket.co.kr/4272567157/still/300?ver=1759119735',
  'rank': 1},
 {'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=1819951773',
  'display_name': 'rTG 알티지 오메가3 6개월분(180캡슐) (5%적립+쇼핑백1매) 식물성캡슐 비타민D (~9/30)',
  'original_price': 31640,
  'sale_price': 24900,
  'image_url': '//gdimg.gmarket.co.kr/1819951773/still/300?ver=1758611624',
  'rank': 2},
 {'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=2933225337',
  'display_name': '(15%) 프로바이오틱스 생유산균 골드 240포(+10포 추가증정) 총 250포 (9/29(월) 결제건만 증정)',
  'original_price': 25640,
  'sale_price': 19800,
  'image_url': '//gdimg.gmarket.co.kr/2933225337/still/300?ver=1759058487',
  'rank': 3},
 {'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=

## step2. 제품의 각 상세 페이지에서 브랜드병 가져오기
- 혹시 모르니까 rating 값도 가져오는 방법 확인하기
- brand_name
- rating

In [ ]:
import time
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

for item_info in items_info_list :
    driver.get(item_info['item_url'])
    driver.implicitly_wait(10)
    page_source = driver.page_source
    
    # 상세 페이지 html에서 값가져오기
    soup = BeautifulSoup(page_source, 'html.parser')
    brand_ele = soup.select_one('.text__brand .text') # 브랜드명
    if not brand_ele :
        # 브랜드명이 없으면, 등록된 판매자 명 가져오기
        brand_name = soup.find(attrs={'class':'text__seller'}).text
    else :
        brand_name = brand_ele.text
    item_info['brand_name'] = brand_name.strip() if brand_name.strip() != '스타배송' else None
    
    # rating = float(soup.find(attrs={'class':'text__score'}).find(string=True, recursive=False))
    # item_info['rating'] = rating
    time.sleep(1.5)
    
driver.quit()

In [19]:
import json
# JSON 파일로 저장
with open('gmarket_product_list_20250929_1.json', 'w', encoding='utf-8') as f:
    json.dump(items_info_list, f, ensure_ascii=False, indent=4)

In [17]:
items_info_list

[{'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=4272567157',
  'display_name': '활기력 20ml 16병 선물포장 (쇼핑백동봉)',
  'original_price': 42000,
  'sale_price': 29900,
  'image_url': '//gdimg.gmarket.co.kr/4272567157/still/300?ver=1759119735',
  'rank': 1,
  'brand_name': '정관장'},
 {'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=1819951773',
  'display_name': 'rTG 알티지 오메가3 6개월분(180캡슐) (5%적립+쇼핑백1매) 식물성캡슐 비타민D (~9/30)',
  'original_price': 31640,
  'sale_price': 24900,
  'image_url': '//gdimg.gmarket.co.kr/1819951773/still/300?ver=1758611624',
  'rank': 2,
  'brand_name': '순수식품'},
 {'shop_name': 'gmarket',
  'item_url': 'https://item.gmarket.co.kr/Item?goodscode=2933225337',
  'display_name': '(15%) 프로바이오틱스 생유산균 골드 240포(+10포 추가증정) 총 250포 (9/29(월) 결제건만 증정)',
  'original_price': 25640,
  'sale_price': 19800,
  'image_url': '//gdimg.gmarket.co.kr/2933225337/still/300?ver=1759058487',
  'rank': 3,
  'brand_name': '서울약사신협'},
 {'shop_na